In [1]:

import h5py
import matplotlib.pyplot as plt
import numpy as np
import argparse
import importlib
import random
import os
import FLAlgorithms
import utils

#for jupyter notebook reload modules
import imp
imp.reload(FLAlgorithms) 
imp.reload(utils) 
from FLAlgorithms.servers.serveravg import FedAvg
from FLAlgorithms.servers.serverpFedMe import pFedMe
from FLAlgorithms.servers.serverperavg import PerAvg
from FLAlgorithms.trainmodel.models import *
from FLAlgorithms.servers.servertrans import pFedTrans

from utils.plot_utils import *

import torch

torch.manual_seed(0)


In [2]:
gpu = 0
device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")

print("---------------Running time:------------",)
# Generate model
model = 'mclr'
dataset = 'Mnist'

model = Mclr_Logistic().to(device), model
algorithm = "pFedMe"
# select algorithm
batch_size = 20
learning_rate = 0.01
personal_learning_rate = 0.01
beta = 2
lamda = 15
num_glob_iters = 800
local_epochs = 20
optimizer = "SGD"
numusers = 5
K = 5
server = pFedTrans(device, dataset, algorithm, model, batch_size, learning_rate,beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, K, personal_learning_rate, 5)
        

---------------Running time:------------
Number of users / total users: 5  /  20
Finished creating pFedMe server.


In [3]:
server.emb_layer.to(device)
for user in server.users:
    user.train(10)
    user.net_values = [*user.model.state_dict().values()]
    user.per_values = user.net_values[-2:]
    value_vec = nn.utils.parameters_to_vector(user.per_values).clone()
    user.emb_vec = server.emb_layer(value_vec)

In [4]:

import torch
import numpy as np
from kmeans_pytorch import kmeans

In [5]:
server.evaluate()

Average Global Accurancy:  0.9306155507559395
Average Global Trainning Accurancy:  0.9400505597688696
Average Global Trainning Loss:  0.6547107414116107


In [6]:
for user in server.users:
    user.train(server.local_epochs)
    user.emb(server.emb_layer)

In [7]:

# data
data_size, dims, num_clusters = 1000, 100, 10

client_emb_list = [user.emb_vec.data.clone().reshape(1, -1) for user in server.users]
print(len(client_emb_list))

20


In [8]:

client_emb_list = torch.cat(client_emb_list, dim=0)

In [9]:
print(client_emb_list)

tensor([[ 0.0026, -0.0084,  0.0218,  ...,  0.0389, -0.0186,  0.0014],
        [ 0.0058, -0.0061,  0.0197,  ...,  0.0393, -0.0129,  0.0027],
        [ 0.0064, -0.0094,  0.0134,  ...,  0.0399, -0.0117,  0.0021],
        ...,
        [ 0.0069, -0.0079,  0.0153,  ...,  0.0413, -0.0013,  0.0014],
        [ 0.0073, -0.0062,  0.0212,  ...,  0.0393, -0.0086,  0.0040],
        [ 0.0044, -0.0089,  0.0136,  ...,  0.0398, -0.0111,  0.0026]])


In [10]:
print(device)

cpu


In [11]:

num_clusters = 10
# kmeans
cluster_res, cluster_centers = kmeans(
    X=client_emb_list, num_clusters=num_clusters, distance='euclidean', device=device
)
#cluster_res, cluster_centers = kmeans(
#    X=client_emb_list, num_clusters=num_clusters, distance='euclidean', device=torch.device('cuda:0')
#)

running k-means on cpu..


[running kmeans]: 2it [00:00, 333.25it/s, center_shift=0.000000, iteration=2, tol=0.000100]


In [12]:

for client_id, cluster_id in enumerate(cluster_res):
    server.users[client_id].cluster_id = cluster_res[client_id] 
    server.clusters[cluster_id].users.append(server.users[client_id])

In [13]:

for cluster in server.clusters:
    cluster.avg_update_base_values()

TypeError: 'NoneType' object is not iterable

In [ ]:

for cluster in server.clusters:
    cluster.update_model(server.emb_layer)

In [ ]:
from reformer_pytorch import *

In [ ]:
x = torch.randn((10, 1, 10)).to(device)
y = torch.randn((10, 1, 10)).to(device)
z = torch.randn((10, 1, 10)).to(device)

embed_dim = 10
num_heads = 5
attn = nn.MultiheadAttention(embed_dim, num_heads).to(device)
opt, weight = attn(x, y, z)

In [ ]:
embed_dim = 128
num_heads = 8
attn = nn.MultiheadAttention(embed_dim, num_heads).to(device)

In [ ]:
#intra_cluster_attn weigh        
emb_dim = attn_dim = 128
# position embedding :weight of total samples in cluster
cluster_emb_list = [cluster.emb_vec.data.clone().reshape(1, -1) for cluster in server.clusters]
x = torch.cat(cluster_emb_list, dim=0).unsqueeze(1)
ln1 = torch.nn.LayerNorm(x.size()).to(device)
x = ln1(x)
print(x.size())
l1 = nn.Linear(emb_dim, attn_dim).to(device)
l2 = nn.Linear(emb_dim, attn_dim).to(device)
a = l1(x)
b = l2(x)
c = torch.zeros_like(a)
output, weight = attn(a, b, c)
print(weight)

In [ ]:
def model_add(models, weights):
    res = copy.deepcopy(models[0])
    for param in res.parameters():
        param.data = torch.zeros_like(param.data)
        param.grad = torch.zeros_like(param.data)
        print(param)
    for model, weight in zip(models, weights):
        for res_param, model_param in zip(res.parameters(), model.parameters()):
            res_param.data += model_param.data.clone() * weight
    
    return res

In [ ]:
import torch
from torch import nn
models = [nn.Linear(10, 10).to(device) for _ in range(5)]
weights = [1,0, 0, 0, 0]
model_a = model_add(models, weights)
print(models[0].state_dict())
print(model_a.state_dict())

In [ ]:
import copy
weight = weight.squeeze(0)
cluster_models = [cluster.model.clone() for cluster in server.clusters]
for cluster in server.clusters:
    cluster.model = 
print(weight.size())
print(weight[0][1])

In [ ]:

net_values = [*model.state_dict().values()]
base_values = net_values[:-2]
per_values = net_values[-2:]



In [ ]:

ratio = 0.5
base_values = 
for v in base_values
v = base_values * ratio
print(v)

In [ ]:
model = server.clusters[0].model
model_list = []
for param in model.parameters():
    param.grad = param.data
print(value)

In [ ]:
emb_dim = 128
cluster_model_list = [cluster.model.data.clone().reshape(1, -1) for cluster in server.clusters]
attn_dim = 128
x = torch.randn((10, 1, 128)).to(device)
print(x.size())
l1 = nn.Linear(emb_dim, attn_dim).to(device)
l2 = nn.Linear(emb_dim, attn_dim).to(device)

a = l1(x)
b = l2(x)
c = torch.ones_like(a)
output, weight = attn(a, b, c)

print(weight)
torch.einsum('')

In [ ]:
inter_query = inter_query_weight(inter_attn_input)
inter_key = inter_key_weight(inter_attn_input)
inter_value = torch.ones_like(inter_query).to(device)
#inter_key = inter_key_weight(inter_attn_input)
#inter_value = inter_value_weight(inter_query)
attn_output, attn_weight = attn(inter_query, inter_key, inter_value)

In [ ]:
print(attn_weight)

In [ ]:
print(weight)

In [ ]:
z = torch.ones((10, 1, 128)).to(device)
output, weight = attn(x, y, z)
print(weight)

In [ ]:
z = torch.ones((10, 1, 128)).to(device)
attn_output, attn_weight = attn(x, y, z)
print(attn_output)
print(attn_weight)

In [ ]:
torch.rand

In [ ]:
print(dot)

In [ ]:
gpu = 0
device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")

for i in range(5):
    print("---------------Running time:------------",i)
    # Generate model
    model = 'mclr'
    dataset = 'Mnist'

    model = Mclr_Logistic().to(device), model
    algorithm = "pFedMe"
    # select algorithm
    batch_size = 20
    learning_rate = 0.01
    personal_learning_rate = 0.01
    beta = 2
    lamda = 15
    num_glob_iters = 800
    local_epochs = 20
    optimizer = "SGD"
    numusers = 5
    K = 5
    if(algorithm == "FedAvg"):
        server = FedAvg(device, dataset, algorithm, model, batch_size, learning_rate, beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, i)
        
    if(algorithm == "pFedMe"):
        server = pFedMe(device, dataset, algorithm, model, batch_size, learning_rate, beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, K, personal_learning_rate, i)

    if(algorithm == "PerAvg"):
        server = PerAvg(device, dataset, algorithm, model, batch_size, learning_rate, beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, i)

    server.train()
    server.test()


In [ ]:
import torch
import numpy as np
from kmeans_pytorch import kmeans

# data
data_size, dims, num_clusters = 1000, 100, 3
x = np.random.randn(data_size, dims) / 6
x = torch.from_numpy(x)

# kmeans
cluster_ids_x, cluster_centers = kmeans(
    X=x, num_clusters=num_clusters, distance='euclidean', device=torch.device('cuda:0')
)

In [ ]:

print(cluster_ids_x, cluster_centers.size())

In [ ]:
times = 5

average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms="pFedMe_p", batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)
average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms=algorithm, batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)

In [ ]:

user = server.users[0]
model = user.model
net_values = [*model.state_dict().values()]
print(net_values)

In [ ]:
print(server.beta)

In [ ]:

for x, y in server.users[0].testloaderfull:
    x = x.to(user.device)
    print(x.size())
    

In [ ]:

# Average data 
if(algorithm == "PerAvg"):
    algorithm == "PerAvg_p"
if(algorithm == "pFedMe"):
    average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms="pFedMe_p", batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)
average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms=algorithm, batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)

